In [1]:
import numpy as np 
import pandas as pd
import os
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model
import keras
from PIL import Image

In [30]:
gait_label = {"001":1, "002":2, "003":3, "004":4, "005":5, "006":6}
face_label = {"Aayush" : 1, "Ajita" : 2, "Aradhana" : 3, "Ashish" : 4, "Komal": 5, "Rudra" : 6}

In [3]:
%cd /content/drive/MyDrive/Manish Sir Project/Main 

/content/drive/MyDrive/Manish Sir Project/Main


In [4]:
ls

'Copy of facenet_keras_weights.h5'   Gait_split/
'Copy of GEI(SOA_models).ipynb'     'Gait Split'/
'create dataframe.ipynb'            'Implementing Facenet.ipynb'
 Encodings/                          Masked_Data.npz
 Face_dataset/                       model.png
 face_encoder_train.png              Test/
 Facenet_1.ipynb                     Testing_masked/
 Facenet_2.ipynb                     Testing_Unmasked/
 facenet_keras.h5                    Train/
 Faces.ipynb                         Train_Data.npz
 featureExtractor.png                Training/
 GaitReco_Facenet.ipynb              Val/
 Gait_Resnet.ipynb                   Val_Data.npz


In [5]:
gait_train_path_000 = "Gait Split/000/train" 
gait_train_path_054 = "Gait Split/054/train"
gait_train_path_090 = "Gait Split/090/train"
face_train_path = "Face_dataset/train"

In [6]:
def extract(file, required_size = (160, 160)):
  img = Image.open(file)
  img = img.convert("RGB")
  pixels = asarray(img)
  image = Image.fromarray(pixels)
  image = image.resize(required_size)
  face_array = asarray(image)
  return face_array

In [20]:
def generate_embedding(model, img_pixels):
  #converting to float values
  img_pixels = img_pixels.astype('float32')
  mean, std_dev = img_pixels.mean(), img_pixels.std()
  img_std = (img_pixels - mean) / std_dev
  samples = expand_dims(img_pixels, axis = 0)
  # passing through ZF net to produce embeddings
  emb = encoder_model.predict(samples)
  return emb[0]

In [8]:
encoder_model = load_model("facenet_keras.h5" , compile = False)

In [33]:
#def extract_embeddings(path, required_size = (160, 160)):
#train_paths = [face_train_path, gait_train_path_000, gait_train_path_054, gait_train_path_090]
def get_embedding(train_path, label):
  train_dict = { "1" : [], "2" : [], "3" : [], "4" : [], "5" : [], "6" : []}
  #for path in train_paths :
  classes = os.listdir(train_path)
  
  for cls in classes:
    cls_path = os.path.join(train_path, cls)
    print("working for class : ", cls)
    images = os.listdir(cls_path)
    print("No of images :", len(images))
    
    image_list = []
    for image in images:
      image_path = os.path.join(cls_path, image)
      #print(image_path)
      image_array = extract(image_path)
      image_list.append(image_array)
      #print(image_array)
    data = asarray(image_list)
    for img in data:
      embedding = generate_embedding(encoder_model, img)
      emb = asarray(embedding)
      train_dict[str(label[cls])].append(emb)    
  return train_dict

In [34]:
face_train_dict = get_embedding(face_train_path, face_label)

working for class :  Aayush
No of images : 30
working for class :  Ajita
No of images : 30
working for class :  Komal
No of images : 30
working for class :  Ashish
No of images : 30
working for class :  Aradhana
No of images : 30
working for class :  Rudra
No of images : 30


In [38]:
gait_train_dict_000 = get_embedding(gait_train_path_000, gait_label)

working for class :  001
No of images : 30
working for class :  002
No of images : 30
working for class :  005
No of images : 30
working for class :  003
No of images : 30
working for class :  004
No of images : 30
working for class :  006
No of images : 30


In [39]:
gait_train_dict_054 = get_embedding(gait_train_path_054, gait_label)

working for class :  006
No of images : 30
working for class :  004
No of images : 30
working for class :  002
No of images : 30
working for class :  001
No of images : 30
working for class :  005
No of images : 30
working for class :  003
No of images : 30


In [40]:
gait_train_dict_090 = get_embedding(gait_train_path_090, gait_label)

working for class :  002
No of images : 30
working for class :  001
No of images : 30
working for class :  003
No of images : 30
working for class :  006
No of images : 30
working for class :  005
No of images : 30
working for class :  004
No of images : 30


In [41]:
gait_val_path_000 = "Gait Split/000/val" 
gait_val_path_054 = "Gait Split/054/val"
gait_val_path_090 = "Gait Split/090/val"
face_val_path = "Face_dataset/val"

In [42]:
face_val_dict = get_embedding(face_val_path, face_label)

working for class :  Ashish
No of images : 10
working for class :  Aradhana
No of images : 10
working for class :  Aayush
No of images : 10
working for class :  Rudra
No of images : 10
working for class :  Ajita
No of images : 10
working for class :  Komal
No of images : 10


In [43]:
gait_val_dict_000 = get_embedding(gait_val_path_000, gait_label)

working for class :  003
No of images : 10
working for class :  004
No of images : 10
working for class :  002
No of images : 10
working for class :  001
No of images : 10
working for class :  005
No of images : 10
working for class :  006
No of images : 10


In [44]:
gait_val_dict_054 = get_embedding(gait_val_path_054, gait_label)

working for class :  001
No of images : 10
working for class :  002
No of images : 10
working for class :  003
No of images : 10
working for class :  006
No of images : 10
working for class :  005
No of images : 10
working for class :  004
No of images : 10


In [45]:
gait_val_dict_090 = get_embedding(gait_val_path_090, gait_label)

working for class :  001
No of images : 10
working for class :  006
No of images : 10
working for class :  004
No of images : 10
working for class :  005
No of images : 10
working for class :  003
No of images : 10
working for class :  002
No of images : 10


In [46]:
gait_test_path_000 = "Gait Split/000/test" 
gait_test_path_054 = "Gait Split/054/test"
gait_test_path_090 = "Gait Split/090/test"
face_test_path = "Face_dataset/test"

In [69]:
face_test_dict = get_embedding(face_test_path, face_label)

working for class :  Aayush
No of images : 10
working for class :  Ajita
No of images : 10
working for class :  Komal
No of images : 10
working for class :  Ashish
No of images : 10
working for class :  Aradhana
No of images : 10
working for class :  Rudra
No of images : 10


In [48]:
gait_test_dict_000 = get_embedding(gait_test_path_000, gait_label)

working for class :  002
No of images : 10
working for class :  001
No of images : 10
working for class :  005
No of images : 10
working for class :  006
No of images : 10
working for class :  004
No of images : 10
working for class :  003
No of images : 10


In [49]:
gait_test_dict_054 = get_embedding(gait_test_path_054, gait_label)

working for class :  002
No of images : 10
working for class :  003
No of images : 10
working for class :  001
No of images : 10
working for class :  004
No of images : 10
working for class :  005
No of images : 10
working for class :  006
No of images : 10


In [50]:
gait_test_dict_090 = get_embedding(gait_test_path_090, gait_label)

working for class :  001
No of images : 10
working for class :  002
No of images : 10
working for class :  006
No of images : 10
working for class :  004
No of images : 10
working for class :  003
No of images : 10
working for class :  005
No of images : 10


In [71]:
train_df = {"Face Embedding" : [], "Gait_000" : [], "Gait_054" : [], "Gait_090" : [], "labels" : []}
val_df = {"Face Embedding" : [], "Gait_000" : [], "Gait_054" : [], "Gait_090" : [], "labels" : []}
test_df = {"Face Embedding" : [], "Gait_000" : [], "Gait_054" : [], "Gait_090" : [], "labels" : []}

In [72]:
train_df = {"Face Embedding" : [], "Gait_000" : [], "Gait_054" : [], "Gait_090" : [], "labels" : []}
for cls in face_train_dict:
  for i, emb in enumerate(face_train_dict[cls], 0):
    train_df["Face Embedding"].append(face_train_dict[cls][i])
    train_df["Gait_000"].append(gait_train_dict_000[cls][i])
    train_df["Gait_054"].append(gait_train_dict_054[cls][i])
    train_df["Gait_090"].append(gait_train_dict_090[cls][i])
    train_df["labels"].append(int(cls))

In [73]:
test_df = {"Face Embedding" : [], "Gait_000" : [], "Gait_054" : [], "Gait_090" : [], "labels" : []}
for cls in face_test_dict:
  for i, emb in enumerate(face_test_dict[cls], 0):
    test_df["Face Embedding"].append(face_test_dict[cls][i])
    test_df["Gait_000"].append(gait_test_dict_000[cls][i])
    test_df["Gait_054"].append(gait_test_dict_054[cls][i])
    test_df["Gait_090"].append(gait_test_dict_090[cls][i])
    test_df["labels"].append(int(cls))

In [74]:
val_df = {"Face Embedding" : [], "Gait_000" : [], "Gait_054" : [], "Gait_090" : [], "labels" : []}
for cls in face_val_dict:
  for i, emb in enumerate(face_val_dict[cls], 0):
    val_df["Face Embedding"].append(face_val_dict[cls][i])
    val_df["Gait_000"].append(gait_val_dict_000[cls][i])
    val_df["Gait_054"].append(gait_val_dict_054[cls][i])
    val_df["Gait_090"].append(gait_val_dict_090[cls][i])
    val_df["labels"].append(int(cls))

In [75]:
train_df = pd.DataFrame.from_dict(train_df)
test_df = pd.DataFrame.from_dict(test_df)
val_df = pd.DataFrame.from_dict(val_df)

In [76]:
train_df.to_csv("train.csv")
test_df.to_csv("test.csv")
val_df.to_csv("val.csv")